## 1、Semantic Kernel

「 Semantic Kernel (SK) is a lightweight SDK that lets you easily mix conventional programming languages with the latest in Large Language Model (LLM) AI "prompts" with templating, chaining, and planning capabilities out-of-the-box. 」

1. Semantic Kernel 是微软研发的一个开源的，面向大模型的开发框架（SDK）；
2. 它支持你用不同开发语言（C#/Python/Java）基于 OpenAI API/Azure OpenAI API/Huggingface 开发大模型应用；
3. 它封装了一系列开箱即用的工具，包括：提示词模板、链式调用、规划能力等。


### 1.1、 大语言模型开发框架的价值是什么？
所有开发框架（SDK）的核心价值，都是降低开发、维护成本。

大语言模型开发框架的价值，是让开发者可以更方便地开发基于大语言模型的应用。主要提供三类帮助：

1. 第三方能力抽象。比如 LLM、向量数据库、搜索引擎等
2. 常用工具、方案封装
3. 底层实现封装。比如流式接口、超时重连、异步与并行等

举些通俗的例子：

- 与外部功能解依赖
  - 比如可以随意更换 LLM 而不用大量重构代码
  - 更换三方工具也同理
- 经常变的部分要在外部维护而不是放在代码里
  - 比如 Prompt 模板
- 各种环境下都适用
  - 比如线程安全
- 方便调试和测试
  - 至少要能感觉到用了比不用方便吧
  - 合法的输入不会引发框架内部的报错

### 1.2、SK 的开发进展


~~Semantic Kernel 现在还是未正式发版状态。1.0.0 版预计今年底发布。~~

1. 开发进展见项目介绍：https://github.com/microsoft/semantic-kernel 
2. 目前仅支持C#/Python/Java：https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages
3. 不同版本支持的特性：https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages#available-features-in-each-sdk
4. 文档：https://learn.microsoft.com/en-us/semantic-kernel/overview/
5. 更多生态：https://github.com/geffzhang/awesome-semantickernel

不同语言之间的概念都是相通的。本课程以 Python 版为例。


### 1.3、SK 的生态位


与 LangChain 完全重叠。微软将此技术栈命名为 Copilot Stack。

<img src="copilot-stack.png" alt="SK 的生态位" width="400"/>

解释：

- Plugin extensibility: 插件扩展
- Copilots: AI 助手（副驾驶），例如 GitHub Copilot、Office 365 Copilot、Windows Copilot
- AI orchestration: AI 编排，SK 就在这里
- Foundation models: 基础大模型，例如 GPT-4
- AI infrastructure: AI 基础设施，例如 PyTorch、GPU


### 怎么理解这个 **AI 编排**


SK 是个野心勃勃的项目，它希望：

1. 让开发者更容易的把 LLM 的能力集成到应用中，像调用函数一样简单
2. 让开发者开发的 LLM 能力与应用解耦，高度可复用
3. 让开发者能与微软的整个 Copilot 生态紧密结合，互相提供养料

请带着这个视角，逐步体会后面所讲的知识。


### 1.4、SK 基础架构


<img src="mind-and-body-of-semantic-kernel.png" alt="SK 的架构" width="400"/>

解释：

- Models and Memory: 类比为大脑
- Connectors: 用来连接各种外部服务，类似驱动程序
- Plugins: 用来连接内部技能
- Triggers and actions: 外部系统的触发器和动作，类比为四肢



<div class="alert alert-success">
！！！现阶段我们就这样理解ok
    
**类比：** Semantic Kernel 用 **Kernel** 命名，是因为它确实像个操作系统 kernel，做核心资源调配，各种资源都可以挂在它上。

</div>



## 2、Hello SK！

### 2.1 环境搭建

1. 需要python版本>=3.10, <3.13（文档有说明：https://pypi.org/project/semantic-kernel/ ）

2. 安装 SK 包：`pip install semantic-kernel==1.3.0`

3. 如果安装了新环境中没有jupyter notebook，执行这条命令安装：`pip install notebook`


### 2.2 简单示例

第一段代码是初始化。后面所有代码都要在执行过这段代码后，才能执行。


In [1]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments

import os
import asyncio

# 加载 .env 到环境变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 创建Kernel
kernel = Kernel()

# 添加OpenAIChatCompletion服务
service_id="default"
kernel.add_service(
    OpenAIChatCompletion(
        ai_model_id="gpt-3.5-turbo",
        service_id=service_id,
        api_key=os.getenv("OPENAI_API_KEY")
    )
)

讲一个笑话

In [2]:
# 参数用{{}}标识
joke_prompt = "给我讲个关于{{$input}}的笑话吧"

# 定义function
joke_function = kernel.add_function(
    plugin_name="joke_function",
    function_name="joke_function",
    prompt=joke_prompt
)

# 运行 function 看结果
async def run_function():
    return await kernel.invoke(
        joke_function,
        input="Hello world" # 只有一个入参，可以直接指定
    )

# 注意这里直接使用 await 如果你在本地运行请执行：asyncio.run(run_function())
result = await run_function()
print(result)

为了打破沉默，程序员对世界说："Hello world！"世界回答说："你好，我是世界。"程序员突然想起来忘了加分号，于是他说："不好意思，我忘了加分号。"世界回答说："没关系，我喜欢你这种简单直接的方式。"笑话的主题是程序员经典的Hello world程序，通过这个简单的问候，展示了程序员与世界的奇妙互动。


## 3、Kernel & Service

<img src="kernel.png" alt="Kernel" width="800"/>

### 3.1、Kernel能干什么？

1. Select the best AI service to run the prompt. 选择最佳的service 来运行prompt

2. Build the prompt using the provided prompt template.使用提供的提示模板构建提示

3. Send the prompt to the AI service. 将提示发送到 AI 服务

4. Receive and parse the response. 接收并解析响应

5. Return the response from the LLM to your application. 将LLM的响应返回给你的应用


### 3.2 Service是什么？

1. Service 包括运行应用程序所需的 AI 服务（如chat completion）和其他服务（如日志记录和HTTP请求服务）。

2. 定义service id是为了能在sk中唯一标识一个服务

<img src="services.png" alt="Service" width="800"/>

## 4、Plugins
简单说，plugin 就是一组函数的集合。它可以包含两种函数：

- Semantic Functions - 语义函数，本质是 Prompt Engineering
- Native Functions - 原生函数，类似 OpenAI 的 Function Calling


<div class="alert alert-warning">
<b>注意：</b>Plugins 最初命名为 Skills，后来改为 Plugins。但是无论文档还是代码，可能还有大量的「Skill」遗留，预计到 1.0.0 发布才能清理干净。见到后，就知道两者是一回事就好。
</div>

SK 内置了若干好用的 plugin

加载方法：

```python
from semantic_kernel.core_plugins import xxx
```

它们是：

- [`ConversationSummaryPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/conversation_summary_plugin.py) - 生成对话的摘要
- [`HttpPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/http_plugin.py) - 发出 HTTP 请求，支持 GET、POST、PUT 和 DELETE
- [`MathPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/math_plugin.py) - 加法和减法计算
- [`TextMemoryPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/text_memory_plugin.py) - 保存文本到 memory 中，可以对其做向量检索
- [`TextPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/text_plugin.py) - 把文本全部转为大写或小写，去掉头尾的空格（trim）
- [`TimePlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/time_plugin.py) - 获取当前时间及用多种格式获取时间参数
- [`WaitPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/wait_plugin.py) - 等待指定的时间
- [`WebSearchEnginePlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/web_search_engine_plugin.py) - 在互联网上搜索给定的文本

## 4.1、Semantic Functions

Semantic Functions 是纯用数据（Prompt + 配置文件）定义的，不需要编写任何代码。所以它与编程语言无关，可以被任何编程语言调用。

一个典型的 semantic function 包含两个文件：

- skprompt.txt: 存放 prompt，可以包含参数，还可以调用其它函数
- config.json: 存放配置，包括函数功能，参数的数据类型，以及调用大模型时的参数

举例：根据用户的自然语言指示，生成 Linux 命令

### 4.1.1、skprompt.txt

- 参数用{{}}标识

### 4.1.2、config.json

In [ ]:
{
    "schema": 1,
    "type": "completion",
    "description": "将用户指令转为Linux指令",
    "execution_settings": {
        "default": {
            "max_tokens": 256,
            "temperature": 0,
            "top_p": 0,
            "presence_penalty": 0,
            "frequency_penalty": 0
        }
    },
    "input_variables": [
        {
            "name": "input",
            "description": "用户的指令",
            "default": ""
        }
    ]
}

说明：

- `type` 只有 `"completion"` 和 `"embedding"` 两种
- 视频中的写法和我这里的写法有点不一样，我这里参考最新官方文档写的，视频里的好像也能用


上面两个文件都在 [sk_samples/SamplePlugin/GenerateCommand](sk_samples/SamplePlugin/GenerateCommand/) 目录下。


### 4.1.3、调用 Semantic Functions

In [2]:
# 加载 semantic function。注意目录结构
plugin = kernel.add_plugin(parent_directory="./sk_samples/", plugin_name="SamplePlugin")

# 运行 function 看结果
async def run_function():
    # 运行 GenerateCommand plugin 看结果
    return await kernel.invoke(
        plugin["GenerateCommand"],
        input="将系统日期设为2023-04-01"
    )

result = await run_function()
print(result)

date -s "2023-04-01"


## 4.2、Native Functions

用编程语言写的函数，如果用 SK 的 Native Function 方式定义，就能纳入到 SK 的编排体系，可以被 Planner、其它 plugin 调用。

下面，写一个过滤有害 Linux 命令的函数，和 GenerateCommand 组合使用。

这个函数名是 `verify`。如果输入的命令不在规定范围内，就返回 `非法`，否则返回 `合法`。

它可以放到目录结构中，在 [sk_samples/SamplePlugin/SamplePlugin.py](sk_samples/SamplePlugin/SamplePlugin.py) 里加入。

In [3]:
from semantic_kernel.functions import kernel_function

class CommandVerifier:
    # 1.3.0 版本中使用 kernel_function
    @kernel_function(
        description="检查命令是否合法",
        name="verifyCommand",
    )
    def verify(self, command: str) -> str:
        if ">" in command:
            return "非法"
        parts = command.replace(';', '|').split('|')
        for cmd in parts:
            name = cmd.split(" ")[0]
            if name not in ["ls","cat","head","tail","echo"]:
                return "非法"
        return "合法"

<div class="alert alert-success">
<b>划重点：</b>新版本中使用@kernel_function()标识本地方法，而不是视频中使用的@sk_function()
</div>


In [5]:
# 加载 native function
verify_plugin = kernel.add_plugin(CommandVerifier(), "CommandVerifier")

# 看结果
result = await kernel.invoke(
    verify_plugin["verifyCommand"],
    command='date -s "2023-04-01"'
    # command='ls -l'
)
print(result)

非法


<div class="alert alert-success">
<b>划重点：</b>在 SK 中，Semantic Function 和 Native Function 被 Kernel 平等对待。
</div>


## 4.3、用 KernelArguments 实现多参数 Functions


如果 Function 都只有一个参数，那么只要把参数定义为 `{{$input}}`，就可以按前面的例子来使用，比较直观。`{{$input}}`会默认被赋值。

多参数时，就不能用默认机制了，需要定义 `KernelArguments` 类型的变量。


### 4.3.1、多参数 Semantic Function 的写法


In [5]:
# Prompt 模板
sk_prompt = """
讲一个{{$topic1}}和{{$topic2}}的一句话笑话
"""

# 创建 Semantic Function
joke_function = kernel.add_function(
    function_name="joke_function",
    plugin_name="joke_function",
    prompt=sk_prompt
)

# 创建 KernelArguments，并变量赋值
arguments = KernelArguments(topic1="农夫", topic2="狼")

# 看结果
result = await kernel.invoke(
    function = joke_function,
    arguments = arguments
)
print(result)

农夫对狼说：“你想吃我吗？不如我们一起去捕捉那只绵羊，我会请你吃烤肉！”


### 4.3.2、多参数 Native Function 的写法

使用`Annotated`给入参添加说明

In [17]:
from semantic_kernel.functions import kernel_function, KernelArguments
from typing import Annotated


class Math:
    @kernel_function(
        description="加法",
        name="add",
    )
    def add(self, 
            number1: Annotated[float, "被加数"],
            number2: Annotated[float, "加数"]
            ) -> str:
        return number1 + number2

    @kernel_function(
        description="减法",
        name="minus",
    )
    
    def minus(self, 
            number1: Annotated[float, "被减数"],
            number2: Annotated[float, "减数"]
            ) -> str:
        return number1 - number2

In [18]:
# 加载 native function
math_plugin = kernel.add_plugin(Math(), "Math")

# 创建 KernelArguments，并变量赋值
arguments = KernelArguments(number1=1024, number2=65536)

# 看结果
result = await kernel.invoke(
    function = math_plugin["add"],
    arguments = arguments
)
print(f"加法计算结果：{result}")

result = await kernel.invoke(
    function = math_plugin["minus"],
    arguments = arguments
)
print(f"减法计算结果：{result}")

加法计算结果：66560.0
减法计算结果：-64512.0


## 4.4、函数的嵌套调用

### 4.4.1、Semantic Function 嵌套调用

SK 允许在 Prompt 模板中直接调用一个函数，形如
```
{{NestedSample.VerifyCommand $input}}
```

In [2]:
# 加载 semantic function。注意目录结构
nested_sample = kernel.add_plugin(parent_directory="./sk_samples/", plugin_name="NestedSample")

# 看结果
result = await kernel.invoke(
    nested_sample["GenerateCommand"],
    input="删除当前目录",
    # input="显示 example.txt 文件的内容",
)

print(result)

非法，删除当前目录是一个危险且不可逆的操作


### 4.4.2、Native Function 嵌套调用（选）


**注意：** Native Function 的嵌套调用，本质上就是函数嵌套。官方给的写法是在 Kernel 的设计思想下的实现，观感上非常晦涩。

实际开发中，可以根据个人对 SK 内核与设计理念的理解，自行选择使用以下写法，或使用普通的函数调用的写法。


## 5、Memory

Semantic Kernel将embedding的功能封装到了Memory中，用来存储上下文信息，就好像电脑的内存一样，而LLM就像是CPU一样，我们所需要做的就是从内存中取出相关的信息交给CPU处理就好了。

使用流程：

1. 用 `kernel.add_service()` 添加一个文本向量生成服务
2. 注册一个`VolatileMemoryStore`，可以是内存、文件、向量数据库等
3. 用 `memory.save_information()` 保存信息到`VolatileMemoryStore`
4. 用 `memory.search()` 搜索信息

使用 ChatALL 的 README.md 做数据，使用内存作为 memory store，我们演示下基于文档对话。

### 5.1、初始化 Embedding


In [1]:
import semantic_kernel as sk
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
import os
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin


# 加载 .env 到环境变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 创建 semantic kernel
kernel = Kernel()


chat_service_id="default"

oai_chat_service = OpenAIChatCompletion(
    ai_model_id="gpt-3.5-turbo",
    service_id=chat_service_id,
    api_key=os.getenv("OPENAI_API_KEY")
)
embedding_gen = OpenAITextEmbedding(
    ai_model_id="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY")
)

# 添加OpenAIChatCompletion服务
kernel.add_service(oai_chat_service)

# 添加 embedding 服务
kernel.add_service(embedding_gen)

### 5.2、文本向量化


In [2]:
from semantic_kernel.text import split_markdown_lines

# 使用VolatileMemoryStore存放memory数据
memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)

# 读取文件内容
with open('ChatALL.md', 'r') as f:
    content = f.read()

# 将文件内容分片，单片最大 100 token（注意：SK 的 text split 功能目前对中文支持不如对英文支持得好）
lines = split_markdown_lines(content, 100)

# 将分片后的内容，存入内存
for index, line in enumerate(lines):
    print(line)
    await memory.save_information(collection="chatall", id=index, text=line)

<div align="center">
   <img src="src/assets/logo-cover.png" width=256></img>
   <p><strong>同时与所有 AI 机器人聊天，找到最佳答案</strong></p>

[Deutsch](README_DE-DE.md) | [English](README.md) | [Español](README_ES-ES.md) | [Français](README_FR-FR.md) | [Italian](README_IT-IT.md) | [日本語](README_JA-JP.md) | [한국어](README_KO-KR.
md) | [Русский](README_RU-RU.md) | [Tiếng Việt](README_VI-VN.md) | 简体中文

</div>

## 屏幕截图

![Screenshot](screenshots/screenshot-1.png?raw=true)

## 功能

基于大型语言模型（LLMs）的 AI 机器人非常神奇。然而，它们的行为可能是随机的，不同的机器人在不同的任务上表现也有差异。如果你想获得最佳体验，不要一个一个尝试。ChatALL（中文名：齐叨）可以把一条指令同时发给多个 AI，帮助您发现最好的回答。你需要做的只是[下载、安装](https://github.
com/sunner/ChatALL/releases)
和提问。

### 这是你吗？

ChatALL 的典型用户是：

- 🤠**大模型重度玩家**，希望从大模型找到最好的答案，或者最好的创作
- 🤓**大模型研究者**，直观比较各种大模型在不同领域的优劣
- 😎**大模型应用开发者**，快速调试 prompt，寻找表现最佳的基础模型

### 支持的 AI

| AI 机器人
| 网页访问 | API      | 说明                                     |
| ------------------------------------------------------------------------------ | -------- | -------- | --------------------

### 5.3、向量搜索


In [6]:
result = await memory.search(collection="chatall", query="ChatALL怎么下载？")
print(result[0].text)

拥有可以访问这些 AI 的帐号，或 API token。
2. 与 AI 网站有可靠的网络连接。

## 下载 / 安装

从 https://github.com/sunner/ChatALL/releases 下载

### Windows 系统

直接下载 \*-win.exe 安装文件并运行之。

### macOS 系统

对于苹果硅芯片 Mac（M1，M2 CPU），请下载 \*-mac-arm64.


### 5.4、现在用函数嵌套做一个简单的 RAG

例：基于 ChatALL 的说明文档，做问答

在自定义的 Semantic Function 中，嵌套调用内置的 `TextMemoryPlugin`。


In [11]:
from semantic_kernel.functions import KernelArguments

# 导入内置的 `TextMemoryPlugin`。主要使用它的 `recall()`
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

# 直接在代码里创建 semantic function。真实工程不建议这么做
# 里面调用了 `recall()`
sk_prompt = """
基于下面的背景信息回答问题。如果背景信息为空，或者和问题不相关，请回答"我不知道"。

[背景信息开始]
{{recall $input}}
[背景信息结束]

问题：{{$input}}
回答：
"""
ask = kernel.add_function(
    function_name="chat_with_memory",
    plugin_name="chat",
    prompt = sk_prompt
)

# 提问
result = await kernel.invoke(
    ask,
    KernelArguments(
        collection = "chatall",
        relevance = 0.8,
        input="ChatALL 怎么下载？"
    )
)

print(result)

从 https://github.com/sunner/ChatALL/releases 下载对应系统的安装文件。对于Windows系统，下载 \*-win.exe 文件并运行；对于苹果硅芯片Mac，下载 \*-mac-arm64 文件。


### 5.5、连接其它 VectorDB

Semantic Kernel 目前已与很多主流的向量数据库做了适配

具体参考：https://learn.microsoft.com/en-us/semantic-kernel/memories/vector-db


## 6、Planner

SK 的 Planner 目的是 Agent 开发。只封装了几个基本形式，把更多的探索留给了开发者。

### 6.1、什么是智能体（Agent）

将大语言模型作为一个推理引擎。给定一个任务，智能体自动生成完成任务所需的步骤，执行相应动作（例如选择并调用工具），直到任务完成。

这个多步骤的规划过程，就由 **Planner** 完成。

<img src="agent-overview.png" style="margin-left: 0px" width=800px>

### 6.2、SK Python 提供了~~四~~两种 Planner：

1. `SequentialPlanner`
   - 制定包含一系列步骤的计划，这些步骤通过自定义生成的输入和输出变量相互连接
   - 官方例程：https://github.com/microsoft/semantic-kernel/blob/main/python/samples/concepts/planners/sequential_planner.py
2. `FunctionCallingStepwisePlanner`
   - 类似 OpenAI Function Calling，从 kernel 中已注册的所有 plugin 中找到一个该执行的函数
   - 官方例程：https://github.com/microsoft/semantic-kernel/blob/main/python/samples/concepts/planners/openai_function_calling_stepwise_planner.py


### 6.3、SequentialPlaner案例

In [3]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.core_plugins import MathPlugin, TextPlugin, TimePlugin
from semantic_kernel.planners import SequentialPlanner
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


async def main():
    kernel = Kernel()

    service_id = "gpt-3.5"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id, 
            ai_model_id="gpt-3.5-turbo",
            api_key=os.getenv("OPENAI_API_KEY")
        )
    )

    # 添加plugin
    kernel.add_plugins({"math": MathPlugin(), "time": TimePlugin(), "text": TextPlugin()})

    # 创建 sequential planner.
    planner = SequentialPlanner(service_id=service_id, kernel=kernel)

    # 题问
    ask = "What day of the week is today, all uppercase?"
    
    # 创建 plan
    plan = await planner.create_plan(goal=ask)

    # 执行 plan
    result = await plan.invoke(kernel=kernel)

    # 打印 plan中的每一步
    for step in plan._steps:
        print(step.description, ":", step._state.__dict__)

    print("Expected Answer:")
    print(result)
    """
    Output:
    SUNDAY
    """


if __name__ == "__main__":
    await main()

Get the current day of the week : {'execution_settings': None}
Convert a string to uppercase. : {'execution_settings': None}
Expected Answer:
MONDAY


### 6.4、Planner的优势

想想自己写Funcation Calling的过程：

1、编写Json格式的函数定义文件

2、调用LLM，传入上下文问和函数定义文件

3、解析LLM的响应以确定是否要调用函数

4、如果有函数调用，需要解析参数和在本地调用函数

5、返回函数调用结果，再传给LLM

6、重复步骤2-5

使用SK的Planner可以自动执行循环调用过程，让使用者更加专注于构建解决用户需求所需的插件。

为什么可以做到？

> Today's LLM models are capable of iteratively calling functions to solve a user's need. This is accomplished by creating a feedback loop where the AI can call a function, check the result, and then decide what to do next.

今天的LLM模型能够迭代调用函数来解决用户的需求。这是通过创建一个反馈循环来实现的，其中人工智能可以调用函数、检查结果，然后决定下一步做什么。


## 7、VS Code 插件

这是个 VS Code 的插件，在 VS Code 里可以直接创建和调试 Semantic Function。

安装地址：https://marketplace.visualstudio.com/items?itemName=ms-semantic-kernel.semantic-kernel


## 总结

1. 我是否应该使用开发框架？
2. 什么情况下选择 SK ？

- 如果你经常需要替换不同 LLM 或有大量的 Prompt 调试需求，选择一个开发框架会让生活更容易
- 如果你必须使用 C#/JAVA 技术栈，SK 可能是目前唯一的选择
- 如果你用 Python 技术栈，可以对比一下 LangChain 再做取舍（下节课细讲）
